In [27]:
from pathlib import Path
import numpy as np
import time

In [1]:
file_openai_key = "/Users/yinterian/.keys/openai_key"
key = open(file_openai_key, "r").read().strip()

In [5]:
#!pip install openai

In [4]:
import openai
openai.api_key = key

### Generating animal names

In [7]:
def generate_prompt(animal):
    promp = """Suggest three names for an animal that
    is a superhero.
    Animal: Cat
    Names: Captain Sharpclaw, Agent Fluffball,
           The Incredible Feline
    Animal: Dog
    Names: Ruff the Protector, Wonder Canine, Sir Barks-a-Lot
    Animal: {}
    Names:""".format(animal.capitalize())
    
    return promp

In [8]:
promp = generate_prompt("parrot")
promp

'Suggest three names for an animal that\n    is a superhero.\n    Animal: Cat\n    Names: Captain Sharpclaw, Agent Fluffball,\n           The Incredible Feline\n    Animal: Dog\n    Names: Ruff the Protector, Wonder Canine, Sir Barks-a-Lot\n    Animal: Parrot\n    Names:'

In [10]:
animal = "parrot"
response = openai.Completion.create(
  model="gpt-3.5-turbo-instruct",
  prompt=generate_prompt(animal),
  temperature=0.6
)

In [14]:
response
response["choices"][0]["text"]

' Feathered Avenger, Polly Power, The Mighty Macaw '

In [15]:
def generate_prompt(text):
    promp = """
    You will be provided with unstructured data, 
    and your task is to parse it into CSV format.
    {}""".format(text)
    return promp

In [16]:
text = "There are many fruits that were found on the recently discovered planet Goocrux. There are neoskizzles that grow there, which are purple and taste like candy. There are also loheckles, which are a grayish blue fruit and are very tart, a little bit like a lemon. Pounits are a bright green color and are more savory than sweet. There are also plenty of loopnovas which are a neon pink flavor and taste like cotton candy. Finally, there are fruits called glowls, which have a very sour and bitter taste which is acidic and caustic, and a pale orange tinge to them."

In [18]:
messages = [
    {"role": "user", "content": "Tell me a joke."},
    {"role": "assistant", "content": "Why did the chicken cross the road?"},
    {"role": "user", "content": "I don't know, why did the chicken cross the road?"}
]

In [19]:
def get_completion(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.5, 
    )
    return response.choices[0].message["content"]

In [20]:
get_completion(messages)

'To get to the other side!'

## Getting Sentiment Data
To get the data:  <br>
`wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz`

To unzip run: <br>
`! tar -xvf aclImdb_v1.tar.gz`

In [4]:
ls aclImdb/train

labeledBow.feat  pos/             unsupBow.feat    urls_pos.txt
neg/             unsup/           urls_neg.txt     urls_unsup.txt


In [5]:
def read_file(path):
    f = open(path, "r")
    return f.read()

In [6]:
train_path = Path("aclImdb/train/")

In [7]:
pos_paths = [f for f in (train_path/"pos").iterdir()]
len(pos_paths)

12500

In [8]:
neg_paths = [f for f in (train_path/"neg").iterdir()]
len(neg_paths)

12500

In [9]:
read_file(pos_paths[100])

"Sure, it was cheesy and nonsensical and at times corny, but at least the filmmakers didn't try. While most TV movies border on the brink of mediocrity, this film actually has some redeeming qualities to it. The cinematography was pretty good for a TV film, and Viggo Mortensen displays shades of Aragorn in a film about a man who played by his own rules. Most of the flashback sequences were kind of cheesy, but the scene with the mountain lion was intense. I was kind of annoyed by Jason Priestly's role in the film as a rebellious shock-jock, but then again, it's a TV MOVIE! Despite all of the good things, the soundtrack was atrocious. However, it was nice to see Tucson, Arizona prominently featured in the film."

In [10]:
read_file(neg_paths[100])

"Really an amazing pile of pap! <br /><br />A predictable, slow moving, soul destroying, mind numbing movie to which, slitting your own wrists with a rusty bread knife seems... well, almost necessary.<br /><br />The acting is over done for the thin dialogue, every scene is at least twice as long as it needs to be, the intricate details of how this career is collapsing or that career is rising is just far too dreary and mundane for words. The music would be good if you didn't have to sit through the movie, but really, three good songs is not enough reward for the effort required to watch the movie.<br /><br />Watching this film I prayed to God for narcolepsy or for someone to shoot me.<br /><br />Never, ever, ever again."

In [11]:
pos_text = [(read_file(path), 'pos') for path in pos_paths[:100]] 
pos_text[0]

('For a movie that gets no respect there sure are a lot of memorable quotes listed for this gem. Imagine a movie where Joe Piscopo is actually funny! Maureen Stapleton is a scene stealer. The Moroni character is an absolute scream. Watch for Alan "The Skipper" Hale jr. as a police Sgt.',
 'pos')

In [12]:
neg_text = [(read_file(path), 'neg') for path in neg_paths[:100]] 

In [13]:
# let's shuffle the list
all_text = pos_text + neg_text
np.random.shuffle(all_text)

In [14]:
[t[1] for t in all_text][:10]

['neg', 'neg', 'pos', 'neg', 'neg', 'neg', 'pos', 'pos', 'pos', 'pos']

In [16]:
text = pos_text[0][0]
prompt = f"""
    You will be provided with a review of a movie.  
    If the movie review have a positive sentiment write "pos"
    If the movie review have a negative sentiment write "neg"
    \"\"\"{text}\"\"\" 
    """

In [31]:
prompt

'\n    You will be provided with a review of a movie.  \n    If the movie review have a positive sentiment write "pos"\n    If the movie review have a negative sentiment write "neg"\n    """For a movie that gets no respect there sure are a lot of memorable quotes listed for this gem. Imagine a movie where Joe Piscopo is actually funny! Maureen Stapleton is a scene stealer. The Moroni character is an absolute scream. Watch for Alan "The Skipper" Hale jr. as a police Sgt.""" \n    '

## Ask Chat GPT about the sentiment of this text

In [24]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [32]:
get_completion(prompt)

'pos'

In [56]:
def get_response(prompt):
    failure = True
    failure_count = 0 
    while(failure):
        failure = False
        try:
            response = get_completion(prompt)
        except:
            failure = True
            print("failed")
            time.sleep(60)
        failure_count =+1
        if failure_count > 100:
            return -1
    return response

In [57]:
corect_outputs = 0
incorrects = []
for text, sent in all_text:
    prompt = f"""
    You will be provided with a review of a movie.  
    If the movie review have a positive sentiment write "pos"
    If the movie review have a negative sentiment write "neg"
    \"\"\"{text}\"\"\" 
    """
    response = get_response(prompt)
    print(sent, response)
    if sent == response:
        corect_outputs += 1
    else:
        incorrects.append((text, sent))
        print("incorrect")
    time.sleep(20) #otherwise openai might not like it

pos pos
neg neg
neg pos
incorrect
neg neg
neg neg
neg neg
pos pos
pos pos
pos pos
neg neg
neg neg
neg neg
pos pos
pos pos
neg neg
pos pos
neg neg
pos pos
pos pos
neg neg
neg neg
pos pos
pos neg
incorrect
neg neg
pos pos
pos pos
neg neg
neg neg
neg neg
neg neg
pos pos
pos pos
failed
pos pos
pos pos
neg neg
pos pos
pos pos
neg neg
pos pos
pos pos
pos pos
pos pos
pos pos
pos pos
pos pos
neg neg
pos pos
pos pos
neg neg
neg neg
pos pos
pos pos
neg neg
neg neg
pos pos
neg neg
neg neg
neg neg
neg neg
pos pos
pos pos
pos pos
neg neg
neg neg
neg neg
failed
pos pos
neg neg
neg neg
pos pos
neg neg
pos pos
pos pos
pos pos
neg neg
neg neg
neg neg
neg pos
incorrect
pos pos
pos pos
neg neg
pos pos
neg neg
pos pos
neg neg
pos pos
failed
pos pos
pos pos
neg neg
pos pos
neg neg
pos pos
neg neg
neg neg
pos pos
failed
pos pos
pos pos
neg pos
incorrect
neg neg
neg neg
failed
pos pos
pos pos
pos pos
pos pos
neg neg
pos neg
incorrect
pos pos
neg neg
pos pos
neg neg
pos pos
pos pos
pos pos
pos pos
neg neg
fai

In [60]:
# here is the accuracy on 200 reviews
1 - 5/len(all_text)

0.975

In [61]:
incorrects

[("Working with one of the best Shakespeare sources, this film manages to be creditable to it's source, whilst still appealing to a wider audience.<br /><br />Branagh steals the film from under Fishburne's nose, and there's a talented cast on good form.",
  'neg'),
 ('Sure it was well shot and made, very well shot and made! But the story was just so weak. And the portrayal of Lincoln was even weaker. Not that Henry Fonda wasn\'t good but the character he played was nothing but a loon. Do you mean to tell me that Lincoln was a wise cracking smart ass with no respect of the law or the court. I mean who the hell was he supposed to be? Cousin Vinnie? I mean come on, "I\'ll just call you Jackass then"???? I understand that Ford was going for great funny hero guy but I didn\'t really like this guy at all. He cheats in sports, talks like a real sweet simpleton and does not seem to know were to sit in a courtroom. How am I supposed to take this seriously.<br /><br />The twist was even weaker. 